# 华数杯C题核心计算代码

本notebook提取了四个问题的主要计算代码，去除了异常处理等非必要代码，只保留正常计算时会使用到的逻辑。

## 问题概述
- **第一问**: SPD光谱参数计算（CCT、Duv、Rf、Rg、mel-DER）
- **第二问**: 多通道LED光源遗传算法优化
- **第三问**: 全天候太阳光模拟LED控制策略
- **第四问**: 睡眠实验数据统计分析

## 1. 导入必要的库

In [2]:
# 数据处理和数值计算
import numpy as np
import pandas as pd
from scipy.optimize import minimize, brentq, differential_evolution, minimize_scalar
from scipy.interpolate import interp1d
from scipy import stats
import warnings

# 绘图
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 光学计算
import colour

# 其他
import os
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

# 设置
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial', '微软雅黑', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
print(f"使用colour库版本: {colour.__version__}")

使用colour库版本: 0.4.6


## 2. 第一问主要计算代码

### 核心功能：从SPD光谱数据计算5个关键参数
- 相关色温 (CCT)
- 距离普朗克轨迹的距离 (Duv)  
- 保真度指数 (Rf)
- 色域指数 (Rg)
- 褪黑素日光照度比 (mel-DER)

In [ ]:
class SPDCalculator:
    def __init__(self, cie_data_path="C/CIE_xyz_1931_2deg.csv"):
        """初始化SPD计算器"""
        self.load_cie_data(cie_data_path)
        
    def load_cie_data(self, cie_data_path):
        """加载CIE 1931标准观察者数据"""
        cie_data = pd.read_csv(cie_data_path, header=None, 
                             names=['wavelength', 'x_bar', 'y_bar', 'z_bar'])
        
        self.wavelengths = cie_data['wavelength'].values
        self.x_bar = cie_data['x_bar'].values
        self.y_bar = cie_data['y_bar'].values
        self.z_bar = cie_data['z_bar'].values
        
    def interpolate_spd(self, wavelengths, spd_values, target_wavelengths=None):
        """将SPD数据插值到标准波长范围"""
        if target_wavelengths is None:
            target_wavelengths = self.wavelengths
            
        f = interp1d(wavelengths, spd_values, kind='linear', 
                    bounds_error=False, fill_value=0)
        return f(target_wavelengths)
    
    def spd_to_xyz(self, wavelengths, spd_values):
        """将SPD转换为CIE XYZ色彩空间"""
        spd_interp = self.interpolate_spd(wavelengths, spd_values)
        
        # 计算归一化常数
        k = 100 / np.trapz(spd_interp * self.y_bar, self.wavelengths)
        
        # 计算XYZ
        X = k * np.trapz(spd_interp * self.x_bar, self.wavelengths)
        Y = k * np.trapz(spd_interp * self.y_bar, self.wavelengths)
        Z = k * np.trapz(spd_interp * self.z_bar, self.wavelengths)
        
        return X, Y, Z
    
    def xyz_to_uv(self, X, Y, Z):
        """将XYZ转换为CIE 1960 UCS (u,v)色度坐标"""
        denominator = X + 15*Y + 3*Z
        u = (4*X) / denominator
        v = (6*Y) / denominator
        return u, v
    
    def chebyshev_polynomials(self, T):
        """Chebyshev多项式计算等色温线"""
        # u(T)系数
        u_num = 0.860117757 + 1.54118254e-4*T + 1.28641212e-7*T**2
        u_den = 1 + 8.42420235e-4*T + 7.08145163e-7*T**2
        u_T = u_num / u_den
        
        # v(T)系数
        v_num = 0.317398726 + 4.22806245e-5*T + 4.20481691e-8*T**2
        v_den = 1 - 2.89741816e-5*T + 1.61456053e-7*T**2
        v_T = v_num / v_den
        
        # 计算导数
        du_num = 1.54118254e-4 + 2*1.28641212e-7*T
        du_den_deriv = 8.42420235e-4 + 2*7.08145163e-7*T
        du_T = (du_num * u_den - u_num * du_den_deriv) / (u_den**2)
        
        dv_num = 4.22806245e-5 + 2*4.20481691e-8*T
        dv_den_deriv = -2.89741816e-5 + 2*1.61456053e-7*T
        dv_T = (dv_num * v_den - v_num * dv_den_deriv) / (v_den**2)
        
        return u_T, v_T, du_T, dv_T
    
    def calculate_cct(self, u, v):
        """计算相关色温CCT"""
        def distance_squared(T):
            u_T, v_T, _, _ = self.chebyshev_polynomials(T)
            return (u - u_T)**2 + (v - v_T)**2
        
        result = minimize_scalar(distance_squared, bounds=(1000, 20000), method='bounded')
        return result.x
    
    def calculate_duv(self, u, v, cct):
        """计算距离普朗克轨迹的距离Duv"""
        u_t, v_t, _, _ = self.chebyshev_polynomials(cct)
        sign = 1 if v > v_t else -1
        duv = sign * np.sqrt((u - u_t)**2 + (v - v_t)**2)
        return duv
    
    def calculate_color_rendering(self, wavelengths, spd_values):
        """计算颜色渲染指数Rf和Rg"""
        # 创建colour库兼容的光谱分布
        spd_dict = {}
        for i, wl in enumerate(wavelengths):
            spd_dict[int(wl)] = spd_values[i]
        
        spd = colour.SpectralDistribution(spd_dict, name='Test SPD')
        
        # 使用ANSI/IES TM-30-18方法计算颜色渲染指数
        tm30_results = colour.colour_fidelity_index(
            spd, additional_data=True, method='ANSI/IES TM-30-18')
        
        return float(tm30_results.R_f), float(tm30_results.R_g), float(tm30_results.D_uv), float(tm30_results.CCT)
    
    def _cie_s026_melanopic_function(self, wavelengths):
        """基于CIE S026/E:2018标准的褪黑素效应函数"""
        # 主峰参数 (480nm)
        peak1_center = 480.0
        peak1_amplitude = 1.0
        peak1_width = 25.0
        
        # 次峰参数 (420nm)  
        peak2_center = 420.0
        peak2_amplitude = 0.15
        peak2_width = 20.0
        
        # 双峰高斯函数
        peak1 = peak1_amplitude * np.exp(-((wavelengths - peak1_center)**2) / (2 * peak1_width**2))
        peak2 = peak2_amplitude * np.exp(-((wavelengths - peak2_center)**2) / (2 * peak2_width**2))
        
        s_mel = peak1 + peak2
        s_mel[wavelengths < 380] = 0
        s_mel[wavelengths > 780] = 0
        
        return s_mel
    
    def _standard_photopic_function(self, wavelengths):
        """标准明视觉光效函数 V(λ)"""
        center = 555.0
        width = 80.0
        V_lambda = np.exp(-((wavelengths - center)**2) / (2 * width**2))
        V_lambda[wavelengths < 380] = 0
        V_lambda[wavelengths > 780] = 0
        return V_lambda
    
    def _cie_d65_approximation(self, wavelengths):
        """CIE D65照明体的解析近似"""
        base_spectrum = np.ones_like(wavelengths)
        blue_enhancement = 1 + 0.3 * np.exp(-((wavelengths - 460)**2) / (2 * 60**2))
        red_attenuation = 1 - 0.1 * np.exp(-((wavelengths - 650)**2) / (2 * 80**2))
        
        D65_approx = base_spectrum * blue_enhancement * red_attenuation
        D65_approx[wavelengths < 300] = 0
        D65_approx[wavelengths > 830] = 0
        
        return D65_approx
    
    def calculate_mel_der(self, wavelengths, spd_values):
        """计算褪黑素日光照度比mel-DER"""
        # 定义标准波长范围
        common_wavelengths = np.arange(380, 781, 1)
        
        # 获取各种光效函数
        photopic_interp = self._standard_photopic_function(common_wavelengths)
        D65_interp = self._cie_d65_approximation(common_wavelengths)
        s_mel_interp = self._cie_s026_melanopic_function(common_wavelengths)
        
        # 插值测试SPD到标准波长范围
        spd_interp = self.interpolate_spd(wavelengths, spd_values, common_wavelengths)
        
        # 计算测试光源的褪黑素辐照度
        E_mel = np.trapz(spd_interp * s_mel_interp, common_wavelengths)
        E_phot = np.trapz(spd_interp * photopic_interp, common_wavelengths) * 683
        mel_ELR = (E_mel / E_phot) * 1000 if E_phot > 0 else 0
        
        # 计算D65照明体的褪黑素辐照度
        E_mel_D65 = np.trapz(D65_interp * s_mel_interp, common_wavelengths)
        E_phot_D65 = np.trapz(D65_interp * photopic_interp, common_wavelengths) * 683
        mel_ELR_D65 = (E_mel_D65 / E_phot_D65) * 1000 if E_phot_D65 > 0 else 1
        
        # 计算mel-DER
        mel_DER = mel_ELR / mel_ELR_D65 if mel_ELR_D65 > 0 else 0
        
        return mel_DER
    
    def calculate_all_parameters(self, wavelengths, spd_values):
        """计算所有5个核心参数"""
        results = {}
        
        # 计算XYZ和色度坐标
        X, Y, Z = self.spd_to_xyz(wavelengths, spd_values)
        u, v = self.xyz_to_uv(X, Y, Z)
        
        # 计算颜色渲染指数（TM-30方法同时返回CCT和Duv）
        rf, rg, duv, cct = self.calculate_color_rendering(wavelengths, spd_values)
        results['Rf'] = rf
        results['Rg'] = rg
        results['Duv'] = duv
        results['CCT'] = cct
        
        # 计算mel-DER
        results['mel-DER'] = self.calculate_mel_der(wavelengths, spd_values)
        
        return results

def load_spd_from_excel(file_path, sheet_name=None):
    """从Excel文件加载SPD数据"""
    if sheet_name is None:
        xl = pd.ExcelFile(file_path)
        sheet_name = xl.sheet_names[0]
    
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None, skiprows=1)
    
    wavelengths = []
    spd_values = []
    
    for i in range(len(df)):
        wavelength_str = str(df.iloc[i, 0])
        spd_value = df.iloc[i, 1]
        
        if pd.isna(spd_value) or wavelength_str == 'nan':
            continue
        
        # 提取波长数值
        if '(' in wavelength_str:
            wavelength = float(wavelength_str.split('(')[0])
        else:
            wavelength = float(wavelength_str)
        
        wavelengths.append(wavelength)
        spd_values.append(float(spd_value))
    
    return np.array(wavelengths), np.array(spd_values)

## 3. 第二问主要计算代码

### 核心功能：多通道LED光源遗传算法优化
- 日间照明模式：最大化Rf，约束CCT在6000±500K
- 夜间助眠模式：最小化mel-DER，约束CCT在3000±500K

In [ ]:
class LEDOptimizer:
    def __init__(self, led_data_path="C/附录2_LED_SPD.xlsx"):
        """初始化LED优化器"""
        self.calculator = SPDCalculator()
        self.load_led_data(led_data_path)
        
    def load_led_data(self, led_data_path):
        """加载5个LED通道的SPD数据"""
        df = pd.read_excel(led_data_path, sheet_name=0, header=0)
        
        # 提取波长信息
        wavelength_col = df.iloc[:, 0]
        self.wavelengths = []
        
        for i, wl_str in enumerate(wavelength_col):
            if pd.isna(wl_str):
                continue
            wl_str = str(wl_str)
            if '(' in wl_str:
                wl = float(wl_str.split('(')[0])
            else:
                wl = float(wl_str)
            self.wavelengths.append(wl)
        
        self.wavelengths = np.array(self.wavelengths)
        
        # 提取5个LED通道的SPD数据
        self.led_channels = {}
        channel_names = ['Blue', 'Green', 'Red', 'Warm White', 'Cold White']
        
        for ch_name in channel_names:
            if ch_name in df.columns:
                spd_data = df[ch_name].values[:len(self.wavelengths)]
                spd_data = pd.to_numeric(spd_data, errors='coerce')
                spd_data = np.nan_to_num(spd_data, 0)
                self.led_channels[ch_name] = spd_data
    
    def synthesize_spd(self, weights):
        """合成光谱：各通道SPD的加权线性叠加"""
        channel_names = ['Blue', 'Green', 'Red', 'Warm White', 'Cold White']
        synthesized_spd = np.zeros_like(self.wavelengths, dtype=float)
        
        for i, ch_name in enumerate(channel_names):
            if i < len(weights) and ch_name in self.led_channels:
                synthesized_spd += weights[i] * self.led_channels[ch_name]
        
        return self.wavelengths, synthesized_spd
    
    def objective_function_daylight(self, weights):
        """日间照明模式的目标函数：最大化Rf，约束CCT=6000±500K"""
        weights = np.array(weights)
        if np.sum(weights) == 0:
            return 1000
        weights = weights / np.sum(weights)
        
        wavelengths, spd = self.synthesize_spd(weights)
        if np.sum(spd) == 0:
            return 1000
        
        results = self.calculator.calculate_all_parameters(wavelengths, spd)
        
        cct = results['CCT']
        rf = results['Rf']
        rg = results['Rg']
        
        # 基础目标函数：强化对Rf=100的追求
        rf_clamped = max(0, min(100, rf))
        
        # 多段非线性函数，对接近100的Rf给予指数级奖励
        if rf_clamped >= 98:
            base_objective = 0.01 * (100 - rf_clamped) ** 3  # 立方函数
        elif rf_clamped >= 95:
            base_objective = 0.1 + 0.05 * (98 - rf_clamped) ** 2  # 平方函数
        elif rf_clamped >= 90:
            base_objective = 0.5 + 0.1 * (95 - rf_clamped)  # 线性增长
        else:
            base_objective = 1.0 + 0.2 * (90 - rf_clamped)  # 大幅惩罚
        
        # 约束违反的惩罚项
        penalty = 0
        
        # CCT约束：6000±500K
        cct_target = 6000
        cct_tolerance = 500
        if cct < (cct_target - cct_tolerance) or cct > (cct_target + cct_tolerance):
            cct_deviation = abs(cct - cct_target) - cct_tolerance
            penalty += 8.0 * (cct_deviation / 1000.0)
        
        # Rg约束：95-105
        if rg < 95:
            penalty += 2.0 * (95 - rg) / 10.0
        elif rg > 105:
            penalty += 2.0 * (rg - 105) / 10.0
        
        # Rf最低要求约束：>88
        if rf < 88:
            penalty += 15.0 * (88 - rf) / 10.0
        
        # 计算最终目标函数值
        total_objective = base_objective + penalty
        
        # 超级奖励机制：对极高Rf给予巨大奖励
        if penalty <= 0.5:  # 约束基本满足
            if rf >= 99:
                reward = min(0.95, 0.5 * (rf - 95))
                total_objective = max(-1.0, base_objective - reward)
            elif rf >= 96:
                reward = min(0.8, 0.3 * (rf - 92))
                total_objective = max(-1.0, base_objective - reward)
            elif rf >= 92:
                reward = min(0.5, 0.1 * (rf - 88))
                total_objective = max(-1.0, base_objective - reward)
            else:
                total_objective = max(-1.0, total_objective)
        else:
            total_objective = max(-1.0, total_objective)
        
        return total_objective
    
    def objective_function_night(self, weights):
        """夜间助眠模式的目标函数：最小化mel-DER，约束CCT=3000±500K"""
        weights = np.array(weights)
        if np.sum(weights) == 0:
            return 1000
        weights = weights / np.sum(weights)
        
        wavelengths, spd = self.synthesize_spd(weights)
        if np.sum(spd) == 0:
            return 1000
        
        results = self.calculator.calculate_all_parameters(wavelengths, spd)
        
        cct = results['CCT']
        rf = results['Rf']
        mel_der = results['mel-DER']
        
        # 基础目标函数：mel-DER标准化
        mel_der_clamped = max(0, min(2.0, mel_der))
        base_objective = 2.0 * mel_der_clamped
        
        # 约束违反的惩罚项
        penalty = 0
        
        # CCT约束：3000±500K
        cct_target = 3000
        cct_tolerance = 500
        if cct < (cct_target - cct_tolerance) or cct > (cct_target + cct_tolerance):
            cct_deviation = abs(cct - cct_target) - cct_tolerance
            penalty += 8.0 * (cct_deviation / 1000.0)
        
        # Rf约束：≥80
        if rf < 80:
            penalty += 10.0 * (80 - rf) / 10.0
        
        # 计算最终目标函数值
        total_objective = base_objective + penalty
        
        # 奖励机制：对低mel-DER值给予奖励
        if penalty == 0 and mel_der <= 0.3:
            reward = min(0.9, 0.5 * (0.3 - mel_der))
            total_objective = max(-1.0, base_objective - reward)
        else:
            total_objective = max(-1.0, total_objective)
        
        return total_objective

class GeneticAlgorithm:
    def __init__(self, optimizer, population_size=100, generations=200, 
                 mutation_rate=0.1, crossover_rate=0.7):
        """遗传算法类"""
        self.optimizer = optimizer
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        
    def initialize_population(self):
        """初始化种群"""
        population = []
        for _ in range(self.population_size):
            individual = np.random.random(5)
            population.append(individual)
        return np.array(population)
    
    def evaluate_fitness(self, population, mode='daylight'):
        """评估种群适应度"""
        fitness = []
        for individual in population:
            if mode == 'daylight':
                obj_value = self.optimizer.objective_function_daylight(individual)
            elif mode == 'night':
                obj_value = self.optimizer.objective_function_night(individual)
            
            # 转换为适应度（目标函数值越小，适应度越高）
            fitness.append(1.0 / (1.0 + obj_value))
        
        return np.array(fitness)
    
    def selection(self, population, fitness):
        """轮盘赌选择"""
        selected = []
        total_fitness = np.sum(fitness)
        
        if total_fitness == 0:
            return population[np.random.choice(len(population), size=len(population))]
        
        probabilities = fitness / total_fitness
        
        for _ in range(len(population)):
            r = np.random.random()
            cumsum = 0
            for i, prob in enumerate(probabilities):
                cumsum += prob
                if r <= cumsum:
                    selected.append(population[i].copy())
                    break
            else:
                selected.append(population[-1].copy())
        
        return np.array(selected)
    
    def crossover(self, parent1, parent2):
        """单点交叉"""
        if np.random.random() < self.crossover_rate:
            crossover_point = np.random.randint(1, len(parent1))
            child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
            return child1, child2
        else:
            return parent1.copy(), parent2.copy()
    
    def mutation(self, individual):
        """高斯变异"""
        mutated = individual.copy()
        for i in range(len(mutated)):
            if np.random.random() < self.mutation_rate:
                mutated[i] += np.random.normal(0, 0.1)
                mutated[i] = max(0, min(1, mutated[i]))
        return mutated
    
    def optimize(self, mode='daylight'):
        """运行遗传算法优化"""
        population = self.initialize_population()
        fitness_history = []
        
        for generation in range(self.generations):
            fitness = self.evaluate_fitness(population, mode)
            best_fitness = np.max(fitness)
            fitness_history.append(best_fitness)
            
            # 选择、交叉、变异
            selected_population = self.selection(population, fitness)
            
            new_population = []
            for i in range(0, len(selected_population), 2):
                parent1 = selected_population[i]
                parent2 = selected_population[(i + 1) % len(selected_population)]
                
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutation(child1)
                child2 = self.mutation(child2)
                
                new_population.extend([child1, child2])
            
            population = np.array(new_population[:self.population_size])
        
        # 返回最优解
        final_fitness = self.evaluate_fitness(population, mode)
        best_idx = np.argmax(final_fitness)
        best_weights = population[best_idx]
        best_fitness = final_fitness[best_idx]

        return best_weights, best_fitness, fitness_history

## 4. 第三问主要计算代码

### 核心功能：全天候太阳光模拟LED控制策略
使用5通道LED系统模拟从早晨8:30到傍晚19:30的太阳光谱数据

In [ ]:
class SolarSpectrumMimicry:
    def __init__(self, led_data_path="C/附录2_LED_SPD.xlsx", solar_data_path="C/附录3_SUN_SPD.xlsx"):
        """初始化太阳光谱模拟器"""
        self.calculator = SPDCalculator()
        self.load_led_data(led_data_path)
        self.load_solar_data(solar_data_path)
        self.optimization_results = {}
    
    def load_led_data(self, led_data_path):
        """加载5个LED通道的SPD数据"""
        df = pd.read_excel(led_data_path, sheet_name=0, header=0)
        
        # 提取波长信息
        wavelength_col = df.iloc[:, 0]
        self.wavelengths = []
        
        for i, wl_str in enumerate(wavelength_col):
            if pd.isna(wl_str):
                break
            
            if isinstance(wl_str, str):
                wl_clean = wl_str.split('(')[0].strip()
                wl = float(wl_clean)
            elif isinstance(wl_str, (int, float)):
                wl = float(wl_str)
            else:
                continue
            
            if 380 <= wl <= 780:
                self.wavelengths.append(wl)
            else:
                break
        
        self.wavelengths = np.array(self.wavelengths)
        
        # 提取5个LED通道的SPD数据
        self.led_channels = {}
        channel_names = ['蓝光', '绿光', '红光', '暖白光(WW)', '冷白光(CW)']
        column_names = ['Blue', 'Green', 'Red', 'Warm White', 'Cold White']
        
        for i, (channel_name, col_name) in enumerate(zip(channel_names, column_names)):
            spd_data = df[col_name].iloc[:len(self.wavelengths)].values
            spd_data = pd.to_numeric(spd_data, errors='coerce')
            spd_data = np.nan_to_num(spd_data, 0)
            self.led_channels[channel_name] = spd_data
    
    def load_solar_data(self, solar_data_path):
        """加载太阳光谱时间序列数据"""
        df = pd.read_excel(solar_data_path, sheet_name=0, header=0)
        
        # 提取波长信息
        wavelength_col = df.iloc[:, 0]
        solar_wavelengths = []
        
        for i, wl_str in enumerate(wavelength_col):
            if pd.isna(wl_str):
                break
            
            if isinstance(wl_str, str):
                wl_clean = wl_str.split('(')[0].strip()
                wl = float(wl_clean)
            elif isinstance(wl_str, (int, float)):
                wl = float(wl_str)
            else:
                continue
            
            if 380 <= wl <= 780:
                solar_wavelengths.append(wl)
            else:
                break
        
        solar_wavelengths = np.array(solar_wavelengths)
        
        # 插值到LED波长网格
        self.solar_spectra = {}
        time_columns = df.columns[1:]
        
        for col in time_columns:
            if hasattr(col, 'strftime'):
                time_label = col.strftime('%H:%M')
            else:
                time_label = str(col)
            
            spectrum_data = df[col].iloc[:len(solar_wavelengths)].values
            spectrum_data = pd.to_numeric(spectrum_data, errors='coerce')
            
            valid_mask = ~np.isnan(spectrum_data) & ~np.isnan(solar_wavelengths)
            if np.sum(valid_mask) < 10:
                continue
                
            valid_wavelengths = solar_wavelengths[valid_mask]
            valid_spectrum = spectrum_data[valid_mask]
            
            if len(valid_wavelengths) > 1:
                interp_func = interp1d(valid_wavelengths, valid_spectrum, 
                                     kind='linear', bounds_error=False, fill_value=0)
                interpolated_spectrum = interp_func(self.wavelengths)
                
                if np.max(interpolated_spectrum) > 0:
                    interpolated_spectrum = interpolated_spectrum / np.max(interpolated_spectrum)
                
                self.solar_spectra[time_label] = interpolated_spectrum
    
    def synthesize_spectrum(self, weights):
        """合成光谱"""
        synthesized = np.zeros_like(self.wavelengths)
        channel_names = ['蓝光', '绿光', '红光', '暖白光(WW)', '冷白光(CW)']
        
        for i, channel_name in enumerate(channel_names):
            synthesized += weights[i] * self.led_channels[channel_name]
        
        return synthesized
    
    def calculate_spectrum_error(self, weights, target_spectrum):
        """计算光谱匹配误差"""
        synthesized = self.synthesize_spectrum(weights)
        error = np.sqrt(np.mean((target_spectrum - synthesized)**2))
        return error
    
    def calculate_parameter_errors(self, weights, target_params):
        """计算关键参数匹配误差"""
        synthesized = self.synthesize_spectrum(weights)
        
        synthetic_params = self.calculator.calculate_all_parameters(
            self.wavelengths, synthesized)
        
        cct_error = 0
        mel_der_error = 0
        
        if 'CCT' in target_params and target_params['CCT'] > 0:
            cct_error = abs(target_params['CCT'] - synthetic_params['CCT']) / target_params['CCT']
        
        if 'mel-DER' in target_params and target_params['mel-DER'] > 0:
            mel_der_error = abs(target_params['mel-DER'] - synthetic_params['mel-DER']) / target_params['mel-DER']
        
        return cct_error + mel_der_error
    
    def optimize_single_timepoint(self, target_spectrum, time_label, alpha=1.0, beta=0.5):
        """优化单个时间点的LED权重"""
        normalized_spectrum = np.maximum(target_spectrum, 0)
        if np.max(normalized_spectrum) > 0:
            normalized_spectrum = normalized_spectrum / np.max(normalized_spectrum)
        
        target_params = self.calculator.calculate_all_parameters(
            self.wavelengths, normalized_spectrum)
        
        def objective(weights):
            spectrum_error = self.calculate_spectrum_error(weights, target_spectrum)
            param_error = self.calculate_parameter_errors(weights, target_params)
            return alpha * spectrum_error + beta * param_error
        
        def weight_constraint(weights):
            return np.sum(weights) - 1.0
        
        constraints = [{'type': 'eq', 'fun': weight_constraint}]
        bounds = [(0, 1) for _ in range(5)]
        
        best_result = None
        best_error = float('inf')
        
        # 多次优化以获得更好结果
        for _ in range(10):
            initial_weights = np.random.rand(5)
            initial_weights = initial_weights / np.sum(initial_weights)
            
            result = minimize(objective, initial_weights, 
                            method='SLSQP', 
                            bounds=bounds, 
                            constraints=constraints,
                            options={'maxiter': 1000, 'ftol': 1e-9})
            
            if result.success and result.fun < best_error:
                best_error = result.fun
                best_result = result
        
        if best_result is None:
            weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        else:
            weights = best_result.x
            weights = weights / np.sum(weights)
        
        synthesized_spectrum = self.synthesize_spectrum(weights)
        spectrum_rmse = np.sqrt(np.mean((target_spectrum - synthesized_spectrum)**2))
        
        synthetic_params = self.calculator.calculate_all_parameters(
            self.wavelengths, synthesized_spectrum)
        
        correlation = np.corrcoef(target_spectrum, synthesized_spectrum)[0, 1]
        if np.isnan(correlation):
            correlation = 0
        
        return {
            'time': time_label,
            'weights': weights,
            'target_spectrum': target_spectrum,
            'synthesized_spectrum': synthesized_spectrum,
            'target_params': target_params,
            'synthetic_params': synthetic_params,
            'spectrum_rmse': spectrum_rmse,
            'correlation': correlation,
            'optimization_error': best_error
        }
    
    def generate_control_sequence(self):
        """生成全天控制序列"""
        control_sequence = []
        
        # 按时间顺序处理每个时间点
        time_points = list(self.solar_spectra.keys())
        
        for i, time_label in enumerate(time_points):
            target_spectrum = self.solar_spectra[time_label]
            
            result = self.optimize_single_timepoint(target_spectrum, time_label)
            control_sequence.append(result)
            self.optimization_results[time_label] = result
        
        # 按时间排序
        control_sequence.sort(key=lambda x: x['time'])
        
        return control_sequence
    
    def analyze_performance(self, control_sequence):
        """分析控制序列性能"""
        if not control_sequence:
            return
        
        # 统计性能指标
        correlations = [result['correlation'] for result in control_sequence]
        rmse_values = [result['spectrum_rmse'] for result in control_sequence]
        
        # 分析CCT跟踪效果
        target_ccts = [result['target_params']['CCT'] for result in control_sequence]
        synthetic_ccts = [result['synthetic_params']['CCT'] for result in control_sequence]
        cct_errors = [abs(t - s) / t for t, s in zip(target_ccts, synthetic_ccts)]
        
        return {
            'correlations': correlations,
            'rmse_values': rmse_values,
            'cct_errors': cct_errors,
            'target_ccts': target_ccts,
            'synthetic_ccts': synthetic_ccts
        }

## 5. 第四问主要计算代码

### 核心功能：基于睡眠实验数据的统计分析
实现睡眠质量评估指标计算和统计分析，验证"夜间助眠模式"的实际效果

In [ ]:
@dataclass
class SleepMetrics:
    """睡眠质量指标数据类"""
    subject_id: int
    condition: str  # 'A', 'B', 'C'
    tst: float      # 总睡眠时间（分钟）
    se: float       # 睡眠效率（%）
    sol: float      # 入睡潜伏期（分钟）
    n3_percent: float   # 深睡眠比例（%）
    rem_percent: float  # REM睡眠比例（%）
    awakenings: int     # 夜间醒来次数

class SleepDataProcessor:
    """睡眠数据处理和分析类"""
    
    def __init__(self, excel_path: str = "C/附录4.xlsx"):
        """初始化睡眠数据处理器"""
        self.excel_path = excel_path
        self.raw_data = None
        self.sleep_metrics = []
        self.conditions_map = {
            0: 'A',  # Night 1: 优化光照（夜间助眠模式）
            1: 'B',  # Night 2: 普通LED光照
            2: 'C'   # Night 3: 黑暗环境
        }
        
    def load_data(self) -> pd.DataFrame:
        """加载睡眠数据"""
        self.raw_data = pd.read_excel(self.excel_path, sheet_name='Problem 4')
        return self.raw_data
    
    def extract_subject_data(self, subject_idx: int) -> List[List[int]]:
        """提取单个被试的三夜睡眠数据"""
        col_start = subject_idx * 3
        subject_data = []
        
        for night in range(3):
            col_idx = col_start + night
            if col_idx < self.raw_data.shape[1]:
                night_data = self.raw_data.iloc[1:, col_idx].dropna().tolist()
                night_data = [int(x) for x in night_data if pd.notna(x) and x in [2, 3, 4, 5]]
                subject_data.append(night_data)
            else:
                subject_data.append([])
        
        return subject_data
    
    def calculate_tst(self, sleep_stages: List[int]) -> float:
        """计算总睡眠时间（分钟）"""
        sleep_epochs = sum(1 for stage in sleep_stages if stage in [2, 3, 5])
        return sleep_epochs * 0.5  # 每个epoch = 30秒 = 0.5分钟
    
    def calculate_sleep_efficiency(self, sleep_stages: List[int]) -> float:
        """计算睡眠效率（%）"""
        tst = self.calculate_tst(sleep_stages)
        tib = len(sleep_stages) * 0.5  # 总卧床时间
        return (tst / tib) * 100 if tib > 0 else 0
    
    def calculate_sol(self, sleep_stages: List[int]) -> float:
        """计算入睡潜伏期（分钟）"""
        for i, stage in enumerate(sleep_stages):
            if stage in [2, 3, 5]:  # 首次进入任何睡眠阶段
                return i * 0.5
        return len(sleep_stages) * 0.5  # 如果整夜未入睡
    
    def calculate_n3_percentage(self, sleep_stages: List[int]) -> float:
        """计算深睡眠比例（%）"""
        n3_time = sum(0.5 for stage in sleep_stages if stage == 3)
        tst = self.calculate_tst(sleep_stages)
        return (n3_time / tst) * 100 if tst > 0 else 0
    
    def calculate_rem_percentage(self, sleep_stages: List[int]) -> float:
        """计算REM睡眠比例（%）"""
        rem_time = sum(0.5 for stage in sleep_stages if stage == 5)
        tst = self.calculate_tst(sleep_stages)
        return (rem_time / tst) * 100 if tst > 0 else 0
    
    def calculate_awakenings(self, sleep_stages: List[int]) -> int:
        """计算夜间醒来次数"""
        if len(sleep_stages) < 2:
            return 0
        
        # 找到入睡时间点
        sol_epoch = None
        for i, stage in enumerate(sleep_stages):
            if stage in [2, 3, 5]:
                sol_epoch = i
                break
        
        if sol_epoch is None:
            return 0
        
        awakenings = 0
        for i in range(sol_epoch, len(sleep_stages) - 1):
            if sleep_stages[i] in [2, 3, 5] and sleep_stages[i + 1] == 4:
                awakenings += 1
        
        return awakenings
    
    def calculate_sleep_metrics(self, sleep_stages: List[int], subject_id: int, condition: str) -> SleepMetrics:
        """计算完整的睡眠质量指标"""
        return SleepMetrics(
            subject_id=subject_id,
            condition=condition,
            tst=self.calculate_tst(sleep_stages),
            se=self.calculate_sleep_efficiency(sleep_stages),
            sol=self.calculate_sol(sleep_stages),
            n3_percent=self.calculate_n3_percentage(sleep_stages),
            rem_percent=self.calculate_rem_percentage(sleep_stages),
            awakenings=self.calculate_awakenings(sleep_stages)
        )
    
    def process_all_data(self) -> List[SleepMetrics]:
        """处理所有被试的睡眠数据"""
        if self.raw_data is None:
            self.load_data()
        
        self.sleep_metrics = []
        n_subjects = min(11, self.raw_data.shape[1] // 3)  # 最多11个被试
        
        for subject_idx in range(n_subjects):
            subject_id = subject_idx + 1
            subject_data = self.extract_subject_data(subject_idx)
            
            for night_idx, night_data in enumerate(subject_data):
                if len(night_data) > 0:
                    condition = self.conditions_map[night_idx]
                    metrics = self.calculate_sleep_metrics(night_data, subject_id, condition)
                    self.sleep_metrics.append(metrics)
        
        return self.sleep_metrics
    
    def get_dataframe(self) -> pd.DataFrame:
        """将睡眠指标转换为DataFrame"""
        if not self.sleep_metrics:
            self.process_all_data()
        
        data = []
        for metrics in self.sleep_metrics:
            data.append({
                'Subject_ID': metrics.subject_id,
                'Condition': metrics.condition,
                'TST': metrics.tst,
                'SE': metrics.se,
                'SOL': metrics.sol,
                'N3_Percent': metrics.n3_percent,
                'REM_Percent': metrics.rem_percent,
                'Awakenings': metrics.awakenings
            })
        
        return pd.DataFrame(data)
    
    def descriptive_statistics(self, df: pd.DataFrame) -> pd.DataFrame:
        """计算描述性统计"""
        numeric_cols = ['TST', 'SE', 'SOL', 'N3_Percent', 'REM_Percent', 'Awakenings']
        stats_df = df.groupby('Condition')[numeric_cols].agg(['mean', 'std', 'median']).round(2)
        return stats_df
    
    def variance_homogeneity_tests(self, df: pd.DataFrame) -> Dict:
        """方差齐性检验"""
        results = {}
        metrics = ['TST', 'SE', 'SOL', 'N3_Percent', 'REM_Percent', 'Awakenings']
        
        for metric in metrics:
            groups = [df[df['Condition'] == cond][metric].values for cond in ['A', 'B', 'C']]
            
            # Levene检验
            levene_stat, levene_p = stats.levene(*groups)
            
            # Bartlett检验
            bartlett_stat, bartlett_p = stats.bartlett(*groups)
            
            results[metric] = {
                'levene_statistic': levene_stat,
                'levene_p_value': levene_p,
                'bartlett_statistic': bartlett_stat,
                'bartlett_p_value': bartlett_p,
                'variance_homogeneous': levene_p > 0.05 and bartlett_p > 0.05
            }
        
        return results
    
    def normality_tests(self, df: pd.DataFrame) -> Dict:
        """正态性检验"""
        results = {}
        metrics = ['TST', 'SE', 'SOL', 'N3_Percent', 'REM_Percent', 'Awakenings']
        
        for metric in metrics:
            results[metric] = {}
            for condition in ['A', 'B', 'C']:
                data = df[df['Condition'] == condition][metric]
                
                # Shapiro-Wilk检验
                shapiro_stat, shapiro_p = stats.shapiro(data)
                
                results[metric][condition] = {
                    'shapiro_statistic': shapiro_stat,
                    'shapiro_p_value': shapiro_p,
                    'is_normal': shapiro_p > 0.05
                }
        
        return results
    
    def perform_anova(self, df: pd.DataFrame) -> Dict:
        """执行方差分析"""
        results = {}
        metrics = ['TST', 'SE', 'SOL', 'N3_Percent', 'REM_Percent', 'Awakenings']
        
        for metric in metrics:
            groups = [df[df['Condition'] == cond][metric].values for cond in ['A', 'B', 'C']]
            f_stat, p_value = stats.f_oneway(*groups)
            results[metric] = {'F_statistic': f_stat, 'p_value': p_value}
        
        return results
    
    def perform_friedman_test(self, df: pd.DataFrame) -> Dict:
        """执行Friedman检验（非参数）"""
        results = {}
        metrics = ['TST', 'SE', 'SOL', 'N3_Percent', 'REM_Percent', 'Awakenings']
        
        for metric in metrics:
            # 重新组织数据为被试×条件格式
            pivot_data = df.pivot(index='Subject_ID', columns='Condition', values=metric)
            pivot_data = pivot_data.dropna()
            
            if len(pivot_data) >= 3:  # 至少需要3个被试
                stat, p_value = stats.friedmanchisquare(
                    pivot_data['A'], pivot_data['B'], pivot_data['C'])
                results[metric] = {'chi2_statistic': stat, 'p_value': p_value}
            else:
                results[metric] = {'chi2_statistic': np.nan, 'p_value': np.nan}
        
        return results
    
    def comprehensive_analysis(self) -> Dict:
        """综合分析"""
        df = self.get_dataframe()
        
        # 方差齐性检验
        variance_results = self.variance_homogeneity_tests(df)
        
        # 正态性检验
        normality_results = self.normality_tests(df)
        
        # 根据假设检验结果选择合适的统计方法
        statistical_results = {}
        
        for metric in ['TST', 'SE', 'SOL', 'N3_Percent', 'REM_Percent', 'Awakenings']:
            # 检查方差齐性和正态性
            variance_ok = variance_results[metric]['variance_homogeneous']
            normality_ok = all(normality_results[metric][cond]['is_normal'] 
                             for cond in ['A', 'B', 'C'])
            
            if variance_ok and normality_ok:
                # 使用ANOVA
                anova_results = self.perform_anova(df)
                statistical_results[metric] = {
                    'method': 'ANOVA',
                    'statistic': anova_results[metric]['F_statistic'],
                    'p_value': anova_results[metric]['p_value']
                }
            else:
                # 使用Friedman检验
                friedman_results = self.perform_friedman_test(df)
                statistical_results[metric] = {
                    'method': 'Friedman',
                    'statistic': friedman_results[metric]['chi2_statistic'],
                    'p_value': friedman_results[metric]['p_value']
                }
        
        return {
            'dataframe': df,
            'descriptive_stats': self.descriptive_statistics(df),
            'variance_tests': variance_results,
            'normality_tests': normality_results,
            'statistical_tests': statistical_results
        }

## 6. 示例使用代码

以下是各个问题核心功能的使用示例

In [ ]:
# 第一问示例：计算SPD参数
def problem1():
    """第一问示例：从SPD数据计算5个关键参数"""
    print("=== 第一问示例：SPD参数计算 ===")
    calculator = SPDCalculator()
    
    # 从Excel文件加载SPD数据
    wavelengths, spd_values = load_spd_from_excel('C/附录1.xlsx')
    
    # 计算所有参数（使用TM-30方法）
    results = calculator.calculate_all_parameters(wavelengths, spd_values)
    
    # 打印结果
    print(f"相关色温 (CCT): {results['CCT']:.1f} K")
    print(f"距离普朗克轨迹的距离 (Duv): {results['Duv']:.4f}")
    print(f"保真度指数 (Rf): {results['Rf']:.1f}")
    print(f"色域指数 (Rg): {results['Rg']:.1f}")
    print(f"褪黑素日光照度比 (mel-DER): {results['mel-DER']:.3f}")
    
    return results

# 第二问示例：LED优化
def problem2():
    """第二问示例：多通道LED光源优化"""
    print("\n=== 第二问示例：LED光源优化 ===")
    optimizer = LEDOptimizer()
    
    # 使用遗传算法优化
    ga = GeneticAlgorithm(optimizer, population_size=100, generations=200)
    
    # 优化日间模式（追求高Rf）
    print("开始日间模式优化...")
    weights_day, fitness_day, history_day = ga.optimize(mode='daylight')
    
    # 分析日间模式结果
    weights_day_norm = weights_day / np.sum(weights_day)
    wavelengths, spd_day = optimizer.synthesize_spd(weights_day_norm)
    results_day = optimizer.calculator.calculate_all_parameters(wavelengths, spd_day)
    
    print(f"日间模式结果:")
    print(f"CCT: {results_day['CCT']:.1f}K, Rf: {results_day['Rf']:.2f}, Rg: {results_day['Rg']:.1f}")
    print(f"目标函数值: {optimizer.objective_function_daylight(weights_day):.4f}")
    
    # 优化夜间模式（最小化mel-DER）
    print("\n开始夜间模式优化...")
    weights_night, fitness_night, history_night = ga.optimize(mode='night')
    
    # 分析夜间模式结果
    weights_night_norm = weights_night / np.sum(weights_night)
    wavelengths, spd_night = optimizer.synthesize_spd(weights_night_norm)
    results_night = optimizer.calculator.calculate_all_parameters(wavelengths, spd_night)
    
    print(f"夜间模式结果:")
    print(f"CCT: {results_night['CCT']:.1f}K, Rf: {results_night['Rf']:.2f}")
    print(f"mel-DER: {results_night['mel-DER']:.4f}")
    print(f"目标函数值: {optimizer.objective_function_night(weights_night):.4f}")
    
    return {
        'weights_day': weights_day,
        'weights_night': weights_night,
        'results_day': results_day,
        'results_night': results_night,
        'history_day': history_day,
        'history_night': history_night
    }

# 第三问示例：太阳光模拟
def problem3():
    """第三问示例：全天候太阳光模拟"""
    print("\n=== 第三问示例：太阳光谱模拟 ===")
    
    mimicry = SolarSpectrumMimicry()
    
    # 生成控制序列
    control_sequence = mimicry.generate_control_sequence()
    
    # 分析性能
    performance = mimicry.analyze_performance(control_sequence)
    
    print(f"处理完成，共优化了{len(control_sequence)}个时间点")
    
    # 展示几个代表性时间点的权重
    if len(control_sequence) >= 3:
        representative_times = [0, len(control_sequence)//2, -1]
        print("\n代表性时间点的LED权重分布:")
        channel_names = ['蓝光', '绿光', '红光', '暖白光', '冷白光']
        
        for idx in representative_times:
            result = control_sequence[idx]
            print(f"{result['time']}: ", end="")
            for i, weight in enumerate(result['weights']):
                print(f"{channel_names[i]}:{weight:.3f} ", end="")
            print(f"(相关性:{result['correlation']:.3f})")
    
    return {
        'control_sequence': control_sequence,
        'performance': performance
    }

# 第四问示例：睡眠数据分析
def problem4():
    """第四问示例：睡眠实验数据分析"""
    print("\n=== 第四问示例：睡眠数据分析 ===")
    
    processor = SleepDataProcessor()
    
    # 综合分析
    results = processor.comprehensive_analysis()
    
    print("睡眠数据处理完成")
    
    # 展示关键统计结果
    df = results['dataframe']
    print(f"数据概览:")
    print(f"总样本数: {len(df)}")
    print(f"被试条件分布:")
    print(df['Condition'].value_counts())
    
    # 展示描述性统计
    print("\n各条件下的平均值:")
    desc_stats = results['descriptive_stats']
    for condition in ['A', 'B', 'C']:
        if condition in desc_stats.index:
            tst_mean = desc_stats.loc[condition, ('TST', 'mean')]
            se_mean = desc_stats.loc[condition, ('SE', 'mean')]
            print(f"条件{condition}: TST={tst_mean:.1f}分钟, SE={se_mean:.1f}%")
    
    # 展示统计检验结果
    print("\n统计检验结果(p值):")
    statistical_results = results['statistical_tests']
    for metric, result in statistical_results.items():
        p_val = result['p_value']
        method = result['method']
        significance = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
        print(f"{metric}({method}): p={p_val:.4f} {significance}")
    
    return results

# 运行所有示例
def run_all_examples():
    """运行所有问题的示例"""
    print("华数杯C题核心计算代码示例运行")
    print("="*50)
    
    # 第一问
    problem1_results = problem1()
    
    # 第二问
    print("\n" + "="*50)
    problem2_results = problem2()
    
    # 第三问
    print("\n" + "="*50)
    problem3_results = problem3()
    
    # 第四问
    print("\n" + "="*50)
    problem4_results = problem4()
    
    print("\n" + "="*50)
    print("所有示例运行完成！")
    
    return {
        'problem1': problem1_results,
        'problem2': problem2_results,
        'problem3': problem3_results,
        'problem4': problem4_results
    }

# 快速测试示例（小规模参数）
def quick_test_examples():
    """快速测试示例（使用较小参数）"""
    print("快速测试模式...")
    
    # 第二问快速测试
    optimizer = LEDOptimizer()
    ga_quick = GeneticAlgorithm(optimizer, population_size=50, generations=50)
    
    print("快速测试日间模式...")
    weights_day, _, _ = ga_quick.optimize(mode='daylight')
    results_day = optimizer.calculator.calculate_all_parameters(
        *optimizer.synthesize_spd(weights_day / np.sum(weights_day)))
    
    print(f"日间模式结果: CCT={results_day['CCT']:.0f}K, Rf={results_day['Rf']:.1f}")
    
    print("快速测试夜间模式...")
    weights_night, _, _ = ga_quick.optimize(mode='night')
    results_night = optimizer.calculator.calculate_all_parameters(
        *optimizer.synthesize_spd(weights_night / np.sum(weights_night)))
    
    print(f"夜间模式结果: CCT={results_night['CCT']:.0f}K, mel-DER={results_night['mel-DER']:.3f}")
    
    return results_day, results_night

# 取消下面的注释来运行示例
# all_results = run_all_examples()  # 完整示例
# quick_results = quick_test_examples()  # 快速测试

## 使用说明

本notebook包含华数杯C题的核心计算代码，包括四个主要问题的解决方案。

### 代码结构：

1. **第一问：SPD参数计算**
   - `SPDCalculator` 类：计算CCT、Duv、Rf、Rg、mel-DER等参数
   - 核心逻辑：基于TM-30方法统一计算CCT和Duv，确保一致性
   - 关键特性：使用colour库的ANSI/IES TM-30-18标准

2. **第二问：LED多通道优化**
   - `LEDOptimizer` 类：5通道LED光源优化
   - `GeneticAlgorithm` 类：遗传算法优化器
   - 核心逻辑：
     - 日间模式：多段非线性函数强化对Rf=100的追求
     - 夜间模式：最小化mel-DER，确保目标函数值≥-1.0
     - 约束处理：CCT约束、Rg约束、超级奖励机制

3. **第三问：太阳光谱模拟**
   - `SolarSpectrumMimicry` 类：全天候太阳光模拟
   - 核心逻辑：
     - 光谱插值和归一化处理
     - 多目标优化（光谱匹配 + 参数匹配）
     - 约束优化求解器（SLSQP方法）

4. **第四问：睡眠数据分析**
   - `SleepDataProcessor` 类：统计分析工具
   - 核心逻辑：
     - 睡眠质量指标计算（TST、SE、SOL、N3%、REM%、醒来次数）
     - 方差齐性检验（Levene、Bartlett）
     - 正态性检验（Shapiro-Wilk）
     - 统计方法选择：ANOVA vs Friedman检验

### 计算逻辑一致性：

**第一问核心改进**：
- 使用TM-30方法统一计算CCT、Duv、Rf、Rg
- mel-DER基于CIE S026标准的双峰高斯函数
- 确保计算结果的标准化和一致性

**第二问核心逻辑**：
- 目标函数采用多段非线性设计，确保数值稳定
- 日间模式：Rf≥98时立方函数，95-98平方函数，<95线性惩罚
- 夜间模式：mel-DER标准化到[0,4]范围，所有结果≥-1.0
- 约束处理：CCT±500K，Rf基础要求，奖励机制

**第三问核心逻辑**：
- LED和太阳光谱数据统一插值处理
- 目标函数：α×光谱误差 + β×参数误差
- 多次随机初始化优化，选择最优解
- 性能评估：相关性、RMSE、CCT跟踪误差

**第四问核心逻辑**：
- 睡眠阶段编码：2(N1), 3(N3), 4(Wake), 5(REM)
- 每epoch = 30秒，TST只计算实际睡眠时间
- 方差齐性+正态性检验决定统计方法
- 被试内设计：Friedman检验 vs 重复测量ANOVA
